In [1]:
import pandas as pd
advertisers = pd.read_json('data/advertisers_dec1.json', lines=True)

In [2]:
buy_trades = pd.read_json('data/buy_trades_dec1.json', lines=True,)
sell_trades = pd.read_json('data/sell_trades_dec1.json', lines=True)

In [3]:
print(len(buy_trades), len(sell_trades), len(advertisers))

13052 45809 30164


In [25]:
adv = pd.concat(
    [pd.json_normalize(advertisers["userDetailVo"], sep="_", max_level=None),
    pd.json_normalize(advertisers["feedback"], sep="_", max_level=None)],
    axis=1)
adv["timestamp"] = advertisers["timestamp"]
adv.sort_values(by=["timestamp"], ascending=False, inplace=True)
adv.drop_duplicates(subset='userNo', keep='first', inplace=True); # unique Users

In [26]:
buy_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
buy_trades.drop_duplicates(subset=["advNo"], keep='first', inplace=True)
sell_trades.sort_values(by=["timestamp"], ascending=False, inplace=True)
sell_trades.drop_duplicates(subset=["advNo"], keep='first', inplace=True)

In [54]:
print(len(buy_trades), len(sell_trades))

3492 6553


In [129]:
import numpy as np
import json

bt = buy_trades["tradeMethods"].to_list()
bex = np.concatenate(bt)
bdf = pd.json_normalize(bex)

st = sell_trades["tradeMethods"].to_list()
sex = np.concatenate(st)
sdf = pd.json_normalize(sex)


In [137]:
bpay_count = bdf.groupby("identifier").count()
sm = bpay_count["tradeMethodName"].sum()
bpay_count["totalPayMethods"] = sm
bpay_count["percentage"] = bpay_count["tradeMethodName"]*100/sm
bpay_count.rename({"identifier": "Pay Method", "tradeMethodName": "Count"}, axis = 1, inplace=True)
bpay_count[[ "Count", "percentage", "totalPayMethods"]].to_csv("csv/trades/method_people_want_to_sell_using.csv")


In [138]:
spay_count = sdf.groupby("identifier").count()
sm = spay_count["tradeMethodName"].sum()
spay_count["totalPayMethods"] = sm
spay_count["percentage"] = spay_count["tradeMethodName"]*100/sm
spay_count.rename({"identifier": "Pay Method", "tradeMethodName": "Count"}, axis = 1, inplace=True)
spay_count[[ "Count", "percentage", "totalPayMethods"]].to_csv("csv/trades/method_people_want_to_buy_with.csv")


In [139]:
bpay_count = bdf.groupby("payType").count()
sm = bpay_count["tradeMethodName"].sum()
bpay_count["totalPayMethods"] = sm
bpay_count["percentage"] = bpay_count["tradeMethodName"]*100/sm
bpay_count.rename({"identifier": "Pay Type", "tradeMethodName": "Count"}, axis = 1, inplace=True)
bpay_count[[ "Count", "percentage", "totalPayMethods"]].to_csv("csv/trades/pay_type_people_want_to_sell_using.csv")
